# TODO

- ~~Convert classes from text to tensor array with size 100~~
- ~~Complete train loop (it is not working now) due to problem above~~
- Find good LSTM architecture
- Check how fast is our approach with per frame check
- ~~Check if we can put videos with different frame number~~

## Web cum test (should we delete it?)

In [4]:
import time
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.PoseModule import PoseDetector


def webcum():
    cap = cv2.VideoCapture(0)
    hands_detector = HandDetector()
    pose_detector = PoseDetector()
    while True:
        success, img = cap.read()
        hands, img1 = hands_detector.findHands(img)
        print(hands)
        img2 = pose_detector.findPose(img)
        lmList, bboxInfo = pose_detector.findPosition(img, bboxWithHands=False)
        print(lmList)
        print()
        time.sleep(2)
        # cv2.imshow("CUM", img)
        # cv2.waitKey(1)

## Device Stuff

In [1]:
import torch

#Set up device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


## Dataset

In [2]:
import json
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [71]:
# Create a dataframe dataset object with all videos from .json config

def create_dataframe(json_config, videos_root):
    data = []
    videos = json.load(open(json_config))
    train, val, test = dict(), dict(), dict()
    for name in os.listdir(videos_root):
        name = name[:-4]
        if videos.get(name) is None:
            continue
        video = videos[name]
        data.append([name, video['subset'], video['action'][1], video['action'][2], video['action'][0]])
    return pd.DataFrame(data, columns=['video_name', 'subset', 'start_frame', 'end_frame', 'class_id'])

dataset = create_dataframe('output3.json', 'wlasl_dataset/videos')
dataset

,video_name,subset,start_frame,end_frame,class_id
0,05727,test,1,74,3
1,05728,train,1,27,3
2,05729,train,1,45,3
3,05730,train,1,32,3
4,05731,train,1,47,3
5,05732,train,1,73,3
6,05733,train,1,92,3
7,05734,train,1,69,3
8,05739,test,1,44,3
9,05740,train,1,42,3


In [72]:
from sklearn.preprocessing import OneHotEncoder

# Train one hot encoder to encoded classes from our dataset
all_classes = dataset["class_id"].to_list()
ohe = OneHotEncoder()
ohe.fit(np.array(all_classes).reshape(-1,1))

# Test
ohe.transform([[14]]).toarray()[0]

array([0., 1.])

In [86]:
from os.path import exists
import torch.nn.functional as F

class ASLDataset(Dataset):
    """
    A custom dataset class for loading American Sign Language (ASL) videos and their corresponding labels.

    Args:
        tensors_folder (str): Path to the folder containing the .pt arrays (converted videos).
        dataset_df (DataFrame): Pandas dataframe with information about videos
        max_frame_count (int): Maximum length of video in frames.
        ohe (OneHotEncoder): Required to encode class id to one hot numeric array.

    Attributes:
        tensors_folder (str): Path to the folder containing the .pt arrays (converted videos).
        name_with_label (dict): A dictionary mapping video names to their labels.
        classes (list): A list of class names.
    """

    def __init__(self, tensors_folder: str, dataset_df: pd.DataFrame, max_frame_count: int, ohe: OneHotEncoder) -> None:
        # Default frame (set of points) to insert in short videos to make all videos equal in length
        default_frame = [0] * (21 * 3 * 2 + 33 * 3)
        
        self.tensor_sequences = []
        self.targets = []
        for index, row in dataset_df.iterrows():
            path_to_tensor = tensors_folder + '/' + row["video_name"] + '.pt'
            
            if not exists(path_to_tensor):
                continue
            
            # Open the video file using OpenCV
            tensor = torch.load(path_to_tensor)

            # Change length of video (not a video but sequence of frames with points coordinates in each frame)
            # pad_right = [default_frame for _ in range(0, max_frame_count - tensor.shape[0])]
            # pad_right = torch.tensor(pad_right)
            # self.tensor_sequences.append(torch.cat((pad_right, F.normalize(tensor.float(), dim=1))))
            self.tensor_sequences.append(F.normalize(tensor.float(), dim=1))
        self.targets = torch.tensor(ohe.transform(np.array(dataset_df["class_id"]).reshape(-1,1)).toarray())
            
            
    def __len__(self):
        """
        Returns the number of videos in the dataset.

        Returns:
            int: The number of videos in the dataset.
        """
        return len(self.targets)

    def __getitem__(self, index):
        """
        Retrieves a video and its corresponding label from the dataset.

        Args:
            index (int): The index of the video to retrieve.

        Returns:
            tuple: A tuple containing the video frames as a PyTorch tensor and the label.
        """

        return self.tensor_sequences[index].float(), self.targets[index].float()


In [87]:
def read_classes(path):
    classes = dict()
    with open(path, 'r') as file:
        for line in file:
            line = line.strip().split('\t')
            key = int(line[0])
            value = line[1]
            classes[key] = value
    return classes

In [88]:
max_frame_count = max(dataset["end_frame"] - dataset["start_frame"] + 1)
print(f"Maximum length of video in frames in current dataset is {max_frame_count}")

Maximum length of video in frames in current dataset is 101


In [89]:
train = dataset[dataset["subset"] == "train"]
#val = dataset[dataset["subset"] == "val"]
test = dataset[dataset["subset"] == "test"]
classes = read_classes('wlasl_dataset/wlasl_class_list.txt')

train_dataset = ASLDataset('point_arrays', train, max_frame_count, ohe)
#val_dataset = ASLDataset('custom_dataset/point_arrays', val, max_frame_count, ohe)
test_dataset = ASLDataset('point_arrays', test, max_frame_count, ohe)

In [90]:
train_dataset[1]

(tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0701,  0.2254, -0.1015],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0665,  0.2097, -0.0235],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0695,  0.2169, -0.0138],
         ...,
         [ 0.0459,  0.0641,  0.0000,  ...,  0.0517,  0.1674,  0.0330],
         [ 0.0481,  0.0671,  0.0000,  ...,  0.0531,  0.1684,  0.0499],
         [ 0.0545,  0.0763,  0.0000,  ...,  0.0578,  0.1838,  0.0345]]),
 tensor([1., 0.]))

In [91]:
# Create dataloaders

batch_size = 1

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train size: {len(train_dataset)}")
#print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 23
Test size: 5


## Model

In [92]:
import torch.nn as nn

class SequenceModel(nn.Module):
    def __init__(self, input_size=1662, output_size=100) -> None:
        super().__init__()
        self.lstm = nn.Sequential(
            nn.LSTM(input_size, 256, 1, batch_first=True), # TODO: Разобраться с LSTM в pytorch
        )
        self.linear = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_size),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:,-1,:])

        return x

In [93]:
model = SequenceModel(input_size=225, output_size=2)
model

SequenceModel(
  (lstm): Sequential(
    (0): LSTM(225, 256, batch_first=True)
  )
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

## Training

In [94]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm


def train_model(model: nn.Module, epochs: int, criterion, optimizer):
    """
    Function that trains model using number of epochs, loss function, optimizer.
    Can use validation or test data set for evaluation.
    Calculates f1 score.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    epochs: int
      Number of train epochs
    criterion
      The loss function from pytorch
    optimizer
      The optimizer from pytorch
    """

    # Train the model
    for epoch in range(epochs):
        model.train()

        #
        predicted_train = []
        true_train = []

        train_loss = 0.0

        bar = tqdm(train_dataloader)
        iterations = 0

        for inputs, outputs in bar:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            predictions = model(inputs.to(device))
            loss = criterion(predictions, outputs.to(device))
            train_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_train.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_train.append(int(torch.argmax(item).cpu().numpy()))
            iterations += 1
            bar.set_postfix(({"loss": f"{train_loss/(iterations*train_dataloader.batch_size)}"}))

        # Computing loss
        train_loss /= len(train_dataset)
        # Computing f1 score
        train_f1 = f1_score(true_train, predicted_train, average="macro")

        # Printing information in the end of train loop
        test_loss, test_f1 = test_model(model, criterion, test_dataloader)
        print(f"Epoch {epoch+1} train (loss: {train_loss:.4f}, f1 score: {train_f1:.4f}) test (loss: {test_loss:.4f}, f1 score: {test_f1:.4f})")


def test_model(model: nn.Module, criterion, test_dataloader: DataLoader):
    """
    Function that evaluates model on specified dataloader
    by specified loss function.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    criterion
      The loss function from pytorch
    test_dataloader: DataLoader
      The dataset for testing model

    Returns
    -------
    float: loss of model on given dataset
    float: f1 score of model on given dataset
    """

    model.eval()

    # Test loss value
    test_loss = 0.0

    # Lists for calculation f1 score
    predicted_test = []
    true_test = []

    with torch.no_grad():
        for inputs, outputs in test_dataloader:

            # Forward pass
            predictions = model(inputs.to(device))
            test_loss += criterion(predictions, outputs.to(device))

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_test.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_test.append(int(torch.argmax(item).cpu().numpy()))

    # Computation of test loss
    test_loss /= len(test_dataloader)

    # Computation of f1 score
    test_f1 = f1_score(true_test, predicted_test, average="macro")
    return test_loss.item(), test_f1


In [95]:
model = SequenceModel(input_size=225, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, 30, criterion, optimizer)

100%|████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 120.01it/s, loss=0.6998955063197924]


Epoch 1 train (loss: 0.6999, f1 score: 0.3030) test (loss: 0.6927, f1 score: 0.3750)


100%|████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 118.31it/s, loss=0.6973083718963291]


Epoch 2 train (loss: 0.6973, f1 score: 0.3429) test (loss: 0.6870, f1 score: 0.3750)


100%|████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 124.91it/s, loss=0.6930680015812749]


...

Epoch 18 train (loss: 0.8350, f1 score: 0.3235) test (loss: 0.9133, f1 score: 0.2857)


100%|████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 130.92it/s, loss=0.8350007469239442]


Epoch 19 train (loss: 0.8350, f1 score: 0.3235) test (loss: 0.9133, f1 score: 0.2857)


100%|████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 127.78it/s, loss=0.8350007585857225]

Epoch 20 train (loss: 0.8350, f1 score: 0.3235) test (loss: 0.9133, f1 score: 0.2857)